# Prueba Chi cuadrada

Para ilustrar este tema utilizaremos el archivo "enigh2020". Importa la biblioteca de *Pandas* y carga esta base de datos en un DataFrame con el nombre *df*

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_excel('https://github.com/adan-rs/AnalisisDatos/raw/main/s02_exploracion_datos/data/enigh2020.xlsx')

## Tabla cruzada

Una forma básica de realizar una tabla cruzada es mediante el método crosstab de Pandas, especificando las variables en las filas y columnas respectivamente. Por ejemplo, para las variables "A" y "B", utilizaremos:  
`pd.crosstab(df['A'], df['B'])`  
El argumento *margins* se puede agregar para mostrar la suma por fila o columna. Cada celda dentro de la tabla mostrará la frecuencia observada en esa intersección

In [ ]:
# Crea una tabla cruzada para relacionar "sexo_jefe" con "est_socio"
tabla = pd.crosstab(df['sexo_jefe'], df['est_socio'], margins=False)
tabla

En muchas ocasiones se requiere representar esta tabla cruzada mediante proporciones. Para ello se puede utilizar el argumento *normalize* con los siguientes valores:

    'all': para dividir cada cantidad entre el total.
    'index': para dividir las celdas en cada fila entre el total de la fila.
    'columns': para dividir las celdas en cada columna entre el total de la columna.

In [ ]:
# Muestra una tabla con las cantidades *normalizadas* entre el total de la columna
tabla_normalizada = pd.crosstab(df['sexo_jefe'], df['est_socio'], normalize='index')
tabla_normalizada

In [ ]:
# Ejemplo de reordenar columnas
orden_columnas = ['bajo', 'medio_bajo', 'medio_alto', 'alto']
tabla_normalizada = tabla_normalizada[orden_columnas]
tabla_normalizada

Para visualizar la relación entre ambas variables se puede utilizar un "mapa de calor". Es recomendable utilizar la librería *seaborn* como en el siguiente ejemplo:  
`ax = sns.heatmap(tabla, annot=True, cmap='YlGnBu', fmt='d')`  
*annot=True* sirve para indicar la cantidad en cada celda, *fmt=d* se utiliza para que se despliegue como números enteros, y *cmap* se utiliza para seleccionar una paleta de colores (ver https://matplotlib.org/stable/gallery/color/colormap_reference.html)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))
ax = sns.heatmap(tabla_normalizada, annot=True, cmap='Greens')

## Prueba Chi cuadrado

*¿Para qué se utiliza?* Se utiliza para evaluar si existe una relación o no entre dos variables cualitativas. 

*Variables*. Se consideran dos variables cualitativas (nominales u ordinales) con relativamente pocas categorías. 

*Tamaño muestral*. En una tabla de 2 x 2, en ninguna casilla la frecuencia esperada debe ser menor que 5. En una tabla de mayor dimensión, no más del 20% de las frecuencias esperadas debe ser menor que 5. 

*Hipótesis*. Las hipótesis son:
- Hipótesis nula (H0): No existe dependencia entre ambas variables (son independientes).
- Hipótesis alternativa (H1): Existe dependencia entre ambas variables.

*Procedimiento*. Para realizar la prueba Chi cuadrada necesitamos importar la biblioteca *scipy.stats*

In [ ]:
import numpy as np
from scipy.stats import chi2_contingency

Un ejemplo de la aplicación a una tabla de contingencia es:  
`chi2, p, dof, expected = chi2_contingency(tabla)`  
Esta instrucción arrojará  los siguientes resultados:
- El estadístico de prueba
- El p-valor
- Los grados de libertad
- La tabla de frecuencias esperadas


Para imprimir estos resultados:  
```
print('Estadístico de prueba:', chi2) 
print('Valor p:', p)  
print('Grados de libertad:', dof)  
print('Tabla de frecuencias esperadas')   
print(expected)
```


In [ ]:
# Realiza una prueba chi cuadrada a la tabla obtenida
chi2, p, dof, expected = chi2_contingency(tabla)

# Mostrar resultados
print('Estadístico de prueba: ', chi2) 
print('Valor p: ', p)  
print('Grados de libertad: ', dof)  
print('Tabla de frecuencias esperadas:')   
print(expected)

In [ ]:
alfa = 0.05
if p <=alfa:
    print('Las variables son dependientes (Se rechaza la hipótesis nula)')
else:
    print('Las variables son independientes (No se rechaza la hipótesis nula)')

Podemos crear una función que integre los pasos anteriores

In [ ]:
from scipy.stats import chi2_contingency

def prueba_chi_cuadrada(df, var1, var2, alfa=0.05, imprimir=True):
    """
    Realiza una prueba chi cuadrada de independencia entre dos variables categóricas.
    Retorna un diccionario con resultados de la prueba.
    """
    # Verificar existencia de variables
    if var1 not in df.columns or var2 not in df.columns:
        raise ValueError(f"Una o ambas variables no existen en el DataFrame: {var1}, {var2}")

    # Crear tabla de contingencia
    tabla_contingencia = pd.crosstab(df[var1], df[var2])

    # Realizar prueba chi cuadrada
    chi2, p_valor, grados_libertad, frecuencias_esperadas = chi2_contingency(tabla_contingencia)

    # Calcular proporción de celdas con frecuencia esperada menor a 5
    celdas_menor_5 = np.sum(frecuencias_esperadas < 5)
    total_celdas = frecuencias_esperadas.size
    prop_freq_menor_5 = celdas_menor_5 / total_celdas

    # Preparar advertencia sobre frecuencias esperadas si es necesario
    advertencia = None
    if prop_freq_menor_5 > 0.2:
        advertencia = ("ADVERTENCIA: Más del 20% de las celdas tienen frecuencias esperadas menores a 5. "
                       "Los resultados de la prueba podrían no ser confiables.")  
    
    # Determinar resultado
    resultado = ('Las variables son dependientes (Se rechaza la hipótesis nula)' 
        if p_valor <= alfa 
        else 'Las variables son independientes (No se rechaza la hipótesis nula)')

    # Imprimir resultados
    print(f'\nTabla de contingencia:\n{tabla_contingencia}\n')
    print(f'Estadístico de prueba: {chi2:.4f}')
    print(f'Valor p: {p_valor:.4f}')
    print(f'Grados de libertad: {grados_libertad}')
    print('\nTabla de frecuencias esperadas:')
    print(pd.DataFrame(
        frecuencias_esperadas,
        index=tabla_contingencia.index,
        columns=tabla_contingencia.columns).round(4))
    print(f'\nProporción de celdas con frecuencia esperada menor a 5: {prop_freq_menor_5:.2%}')
    print(f'({celdas_menor_5} de {total_celdas} celdas)')
    if advertencia:
        print(f'\n{advertencia}')
    print(f'\nNivel de significancia: {alfa}')
    print(f'Resultado: {resultado}')

    # Retornar resultados como diccionario
    return {'tabla_contingencia': tabla_contingencia,
            'chi2': chi2,
            'p_valor': p_valor,
            'grados_libertad': grados_libertad,
            'frecuencias_esperadas': pd.DataFrame(
                frecuencias_esperadas,
                index=tabla_contingencia.index,
                columns=tabla_contingencia.columns).round(4),
            'resultado': resultado}

In [ ]:
resultados = prueba_chi_cuadrada(df, 'sexo_jefe', 'est_socio')

Ejemplo de reporte de metodología y resultados:
>“Se realizó una prueba chi cuadrada para evaluar la relación entre el sexo del jefe de familia y el estrato socioeconómico. Se encontró una relación significativa entre ambas variables, χ²(3) = 12.3959, p = 0.006.”

## Comentarios
La prueba exacta de Fisher es una alternativa apropiada cuando el número de observaciones es pequeño o hay celdas con frecuencias esperadas menores a 5.

## Ejercicio

La base de datos "aerolineas" contiene información de publicaciones en X (antes Twitter) que han sido clasificados por su sentimiento (positivo, neutral y negativo) y que etiquetan a diferentes aerolíneas. Tu tarea es investigar si existe una relación significativa entre el sentimiento de los mensajes y las aerolíneas mencionadas. 

Este análisis te permitirá determinar si la distribución de los sentimientos (positivo, neutral, negativo) difiere entre las diferentes aerolíneas, sugiriendo una posible asociación entre el tipo de sentimiento expresado en los mensajes y la aerolínea mencionada. Los resultados de esta prueba estadística podrían proporcionar información valiosa sobre la percepción del servicio de las distintas aerolíneas entre los usuarios de esta red social.


In [ ]:
df2 = pd.read_excel('data/aerolineas.xlsx')